In [1]:
import re
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K

import gensim
import jamo
from PIL import Image
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm
tqdm.pandas()

from unicode import join_jamos

c:\python37\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
DATA_TYPE = 'full-nouns-jamo-trigram'
DATA_DIR = 'lol'

DIM = 300

In [3]:
MINING_DIR = '1-mining'
TOKENIZING_DIR = '3-tokenizing'
EMBEDDING_DIR = '4-embedding'

mining_dir = Path(MINING_DIR) / DATA_DIR
tokenizing_dir = Path(TOKENIZING_DIR) / DATA_DIR
embedding_dir = Path(EMBEDDING_DIR) / DATA_DIR
embedding_dir.mkdir(parents=True, exist_ok=True)

### data

In [4]:
df = pd.read_csv(embedding_dir / 'news-{}.csv'.format(DATA_TYPE)).dropna()

In [5]:
sentences = [x.split(' ') + y.split(' ') for x, y in zip(df.title, df.contents)]
sentences, _ = train_test_split(sentences, test_size=0.2, shuffle=False)

In [6]:
thumbnails = []
for row in tqdm(df.itertuples(index=False), total=len(df)):
    mining_date_dir = mining_dir / str(row.date)
    thumbnail_glob = '{:04d}-*-thumbnail.jpg'.format(row.ranking)
    try:
        thumbnail_path = next(mining_date_dir.glob(thumbnail_glob))
    except StopIteration:
        thumbnail = np.asarray(Image.new('RGB', (210, 122), (255, 255, 255)))
    else:
        thumbnail = np.asarray(Image.open(thumbnail_path).convert('RGB'))
    thumbnails.append(thumbnail / 255.0)
thumbnails = np.asarray(thumbnails)

KeyboardInterrupt: 

### word2vec, fasttext

In [35]:
wv_vector_path = embedding_dir / 'news-{}-wv{}.vec'.format(DATA_TYPE, DIM)
ft_vector_path = embedding_dir / 'news-{}-ft{}.vec'.format(DATA_TYPE, DIM)

wv_mod = gensim.models.Word2Vec(sentences, sg=1, size=DIM)
wv_mod.wv.save_word2vec_format(str(wv_vector_path))
ft_mod = gensim.models.FastText(sentences, sg=1, size=DIM)
ft_mod.wv.save_word2vec_format(str(ft_vector_path))

In [38]:
ft_mod.wv.word_vec('ㄹㅗㄹㄹㅗㄹㄹㅗㄹ')

array([ 1.42286196e-01,  8.98443982e-02, -1.00184076e-01,  2.86917891e-02,
       -6.66935891e-02,  3.47867720e-02,  1.67990983e-01,  5.65400021e-03,
       -5.91718704e-02, -1.63110822e-01,  2.34631926e-01,  5.78241982e-02,
        2.01397866e-01, -2.01946363e-01, -1.58710137e-01, -1.34217009e-01,
       -5.25152199e-02,  2.44512837e-02, -9.07028560e-03, -1.00660995e-01,
        9.65150669e-02,  1.63656935e-01,  7.28694350e-02,  7.02350885e-02,
       -1.11775156e-02,  1.65030956e-02,  4.42833640e-02, -1.72882646e-01,
       -7.01838285e-02, -5.17754331e-02, -1.40448570e-01,  4.53177914e-02,
        6.13001026e-02, -5.33518456e-02, -5.63084111e-02, -1.82965882e-02,
       -8.66115373e-03, -1.34660779e-02, -5.25258034e-02,  1.21647991e-01,
        9.05212685e-02,  2.16957375e-01,  4.71497700e-02, -4.88047022e-03,
        2.15473488e-01, -1.76374745e-02,  2.17271358e-01, -3.90675813e-02,
       -2.51901329e-01,  1.60450697e-01,  2.19821930e-03, -4.07219566e-02,
       -1.04005253e-02,  